In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from bayes_opt import BayesianOptimization

# Load and prepare the data
data = pd.read_csv('large.csv')  # Replace with your dataset path
pd.set_option('display.max_columns', None)
print(data.head())

# Define the inputs and outputs
X = data.drop(columns=['UCS', 'co2', 'cost'])
y_ucs = data['UCS']
y_co2 = data['co2']
y_cost = data['cost']

# Perform a single train-test split for all target variables, with a consistent random state
X_train, X_test, y_ucs_train, y_ucs_test, y_co2_train, y_co2_test, y_cost_train, y_cost_test = train_test_split(
    X, y_ucs, y_co2, y_cost, test_size=0.2, random_state=42)

# Function to perform K-Fold Cross-Validation
def kfold_cross_val(model, X, y, folds=5):
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    cv_results = cross_validate(model, X, y, cv=kf, scoring='r2', return_train_score=True)
    return cv_results['test_score'], cv_results['train_score']

# Function to optimize XGBoost for a given target
def optimize_xgb(X_train, y_train):
    def xgb_evaluate(n_estimators, learning_rate, max_depth, min_child_weight):
        model = XGBRegressor(
            n_estimators=int(n_estimators),
            learning_rate=learning_rate,
            max_depth=int(max_depth),
            min_child_weight=int(min_child_weight),
            random_state=42
        )
        # Use K-Fold Cross-Validation for consistent evaluation
        r2_scores, _ = kfold_cross_val(model, X_train, y_train, folds=5)
        return np.mean(r2_scores)

    pbounds = {
        'n_estimators': (100, 500),
        'learning_rate': (0.01, 0.3),
        'max_depth': (3, 10),
        'min_child_weight': (1, 10)
    }

    optimizer = BayesianOptimization(f=xgb_evaluate, pbounds=pbounds, random_state=42)
    optimizer.maximize(init_points=5, n_iter=25)
    
    return optimizer.max['params']

# Function to train and evaluate the model with optimized parameters
def train_and_evaluate_model(X_train, X_test, y_train, y_test, target_name):
    print(f"Optimizing {target_name} model...")
    best_params = optimize_xgb(X_train, y_train)
    print(f"Best parameters for {target_name}:", best_params)
    
    model = XGBRegressor(
        n_estimators=int(best_params['n_estimators']),
        learning_rate=best_params['learning_rate'],
        max_depth=int(best_params['max_depth']),
        min_child_weight=int(best_params['min_child_weight']),
        random_state=42
    )
    model.fit(X_train, y_train)

    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Evaluation Metrics
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    mae_test = mean_absolute_error(y_test, y_pred_test)
    
    print(f"{target_name} Model R² (Train): {r2_train:.4f}")
    print(f"{target_name} Model R² (Test): {r2_test:.4f}")
    print(f"{target_name} Model RMSE (Test): {rmse_test:.4f}")
    print(f"{target_name} Model MAE (Test): {mae_test:.4f}")
    print("=" * 50)
    return model

# Train and evaluate each target model and store them in variables
model_ucs = train_and_evaluate_model(X_train, X_test, y_ucs_train, y_ucs_test, 'UCS')
model_co2 = train_and_evaluate_model(X_train, X_test, y_co2_train, y_co2_test, 'CO₂')
model_cost = train_and_evaluate_model(X_train, X_test, y_cost_train, y_cost_test, 'Cost')


   FlyAsh  GGBFS    RM    SM    AM    HM    LM    CA   FA     SS    SH  SS/SH  \
0   476.0    0.0  0.54  1.43  2.44  0.01  0.72  1294  554  120.0  48.0    2.5   
1   476.0    0.0  0.54  1.43  2.44  0.01  0.72  1294  554  120.0  48.0    2.5   
2   476.0    0.0  0.54  1.43  2.44  0.01  0.72  1294  554  120.0  48.0    2.5   
3   476.0    0.0  0.54  1.43  2.44  0.01  0.72  1294  554  120.0  48.0    2.5   
4   476.0    0.0  0.54  1.43  2.44  0.01  0.72  1294  554  120.0  48.0    2.5   

   Mol_SH  SS_SiO2/Na2O  SH_Na2O %   SH_SiO2 %   SH_H2O %    WEff   W/B  \
0     8.0           2.0        14.7        29.4       55.9  99.72  0.35   
1     8.0           2.0        14.7        29.4       55.9  99.72  0.35   
2     8.0           2.0        14.7        29.4       55.9  99.72  0.35   
3     8.0           2.0        14.7        29.4       55.9  99.72  0.35   
4     8.0           2.0        14.7        29.4       55.9  99.72  0.35   

   Ti (hr.)  Cti (°C)  Cm  RH(%)  CTf(°C)  Age(Days)   cost   

In [3]:
# Dictionary to store predictions for each output variable
predictions = {}

# Making predictions for each output variable using the separate models
print("Making predictions on the test set...")

# Predict UCS and store in predictions dictionary
predictions['UCS'] = model_ucs.predict(X_test)
print(f"Predictions for UCS: {predictions['UCS'][:5]}")  # Display the first 5 UCS predictions

# Predict CO₂ and store in predictions dictionary
predictions['co2'] = model_co2.predict(X_test)
print(f"Predictions for CO₂: {predictions['co2'][:5]}")  # Display the first 5 CO₂ predictions

# Predict Cost and store in predictions dictionary
predictions['cost'] = model_cost.predict(X_test)
print(f"Predictions for Cost: {predictions['cost'][:5]}")  # Display the first 5 Cost predictions


Making predictions on the test set...
Predictions for UCS: [54.75744  31.034554  9.223737 30.08334  45.290943]
Predictions for CO₂: [338.16916 358.33255 347.69385 279.80603 413.8644 ]
Predictions for Cost: [79.12113 79.41804 82.77048 68.27971 86.67905]


In [4]:
# Generate predictions for each target using the separate models
ucs_pred = model_ucs.predict(X_test)  # UCS predictions
co2_pred = model_co2.predict(X_test)  # CO₂ predictions
cost_pred = model_cost.predict(X_test)  # Cost predictions

# Check lengths of all arrays
print("Length of UCS_Predicted:", len(ucs_pred))
print("Length of CO2_Predicted:", len(co2_pred))
print("Length of Cost_Predicted:", len(cost_pred))
print("Length of UCS_True:", len(y_ucs_test))
print("Length of CO2_True:", len(y_co2_test))
print("Length of Cost_True:", len(y_cost_test))

# Ensure all lengths match before creating DataFrame
if (len(ucs_pred) == len(co2_pred) == len(cost_pred) == 
    len(y_ucs_test) == len(y_co2_test) == len(y_cost_test)):
    # Create a DataFrame to store predictions alongside true values for comparison
    predictions_df = pd.DataFrame({
        'UCS_Predicted': ucs_pred,
        'co2_Predicted': co2_pred,
        'cost_Predicted': cost_pred,
        'UCS_True': y_ucs_test.values.flatten(),
        'co2_True': y_co2_test.values.flatten(),
        'cost_True': y_cost_test.values.flatten()
    })

    print(predictions_df.head())  # Display the first few rows of predictions and true values
else:
    print("Error: Mismatched array lengths found.")


Length of UCS_Predicted: 668
Length of CO2_Predicted: 668
Length of Cost_Predicted: 668
Length of UCS_True: 668
Length of CO2_True: 668
Length of Cost_True: 668
   UCS_Predicted  co2_Predicted  cost_Predicted  UCS_True    co2_True  \
0      54.757439     338.169159       79.121132        52  338.892000   
1      31.034554     358.332550       79.418037        27  358.452500   
2       9.223737     347.693848       82.770477        11  347.548518   
3      30.083340     279.806030       68.279709        19  281.100000   
4      45.290943     413.864410       86.679047        52  410.254860   

   cost_True  
0  79.196000  
1  79.424000  
2  82.798548  
3  68.600000  
4  86.566600  


In [5]:
import numpy as np
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.operators.sampling.lhs import LHS
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM

# Assuming model_ucs, model_co2, and model_cost have already been trained as separate XGBoost models

# Redefine the MixDesignOptimizationProblem with UCS penalty, using separate models
class MixDesignOptimizationProblem(Problem):
    def __init__(self, bounds, UCS_target, tolerance):
        super().__init__(
            n_var=len(bounds),    # Number of input features
            n_obj=3,              # Objectives: minimize CO2, cost, and UCS deviation
            n_constr=0,           # No hard constraint on UCS
            xl=np.array([bound[0] for bound in bounds.values()]),  # Lower bounds
            xu=np.array([bound[1] for bound in bounds.values()])   # Upper bounds
        )
        self.UCS_target = UCS_target
        self.tolerance = tolerance
    
    def _evaluate(self, X, out, *args, **kwargs):
        # Predictions using separate models
        ucs_pred = model_ucs.predict(X)     # Using the trained UCS model
        co2_pred = model_co2.predict(X)     # Using the trained CO₂ model
        cost_pred = model_cost.predict(X)   # Using the trained cost model

        # Objectives: minimize CO2 and cost, and penalize deviation from UCS target
        ucs_penalty = np.abs(ucs_pred - self.UCS_target)
        out["F"] = np.column_stack([co2_pred, cost_pred, ucs_penalty])

# Define bounds based on your design criteria for the input features
bounds = {
    'FlyAsh': (0, 1530),
    'GGBFS': (0, 100),
    'RM': (0.4, 1),
    'SM': (1, 1.5),
    'AM': (2, 3),
    'HM': (0, 1),
    'LM': (0, 1),
    'CA': (1200, 1300),
    'FA': (500, 600),
    'SS': (100, 130),
    'SH': (0, 1500),
    'SS/SH': (0, 100),
    'Mol_SH': (0, 100),
    'SS_SiO2/Na2O': (0, 1000),
    'SH_Na2O %': (0, 1000),
    'SH_SiO2 %': (0, 1000),
    'SH_H2O %': (0, 1000),
    'WEff': (0, 1000),
    'W/B': (0.3, 0.4),
    'Ti (hr.)': (0, 24),
    'Cti (°C)': (20, 90),
    'Cm': (0, 1),
    'RH(%)': (30, 40),
    'CTf(°C)': (20, 90),
    'Age(Days)': (0, 30),
}

# Define the target UCS and tolerance
UCS_target = 50  # Target UCS
tolerance = 5    # Tolerance for UCS deviation

# Set up NSGA-II with modified parameters
algorithm = NSGA2(
    pop_size=100,
    sampling=LHS(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True,
    pareto_fraction=0.3  # Set Pareto fraction to 0.3
)

# Run the optimization
problem = MixDesignOptimizationProblem(bounds, UCS_target, tolerance)
res = minimize(problem,
               algorithm,
               ('n_gen', 100),  # Number of generations
               verbose=True)

# Display results
print("Best solutions for minimizing CO2 and cost with target UCS:")
if res.X is not None:
    for i in range(len(res.X)):
        print(f"Solution {i+1}:")
        print("Mix Design (Features):", res.X[i])
        print("Predicted CO₂:", res.F[i, 0])
        print("Predicted Cost:", res.F[i, 1])
        
        # Confirm UCS prediction
        ucs_check = model_ucs.predict(res.X[i].reshape(1, -1))
        print("Predicted UCS:", ucs_check[0])
        print("Deviation from UCS_target:", abs(ucs_check[0] - UCS_target))
        print()
else:
    print("No feasible solutions were found by the optimizer.")


n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |      8 |             - |             -
     2 |      200 |     10 |  0.0875914800 |         ideal
     3 |      300 |     13 |  0.0313502668 |         nadir
     4 |      400 |      9 |  0.0055528276 |         ideal
     5 |      500 |     17 |  0.0028455676 |         ideal
     6 |      600 |     15 |  0.0642107495 |         ideal
     7 |      700 |     23 |  0.2430190076 |         nadir
     8 |      800 |     35 |  0.0049877107 |         ideal
     9 |      900 |     38 |  0.0203033527 |             f
    10 |     1000 |     39 |  0.0845859807 |         nadir
    11 |     1100 |     41 |  0.0073119589 |         nadir
    12 |     1200 |     46 |  0.0033855222 |         ideal
    13 |     1300 |     65 |  0.0426825148 |         nadir
    14 |     1400 |     70 |  0.0551154264 |         nadir
    15 |     1500 |     75 |  0.1272054220 |         ideal
    16 |     1600 |     85 |  0.0081476846 |         ide

In [6]:
import numpy as np

# Set target UCS and weights for scoring
UCS_target = 50
cost_weight = 1
co2_weight = 1
ucs_deviation_weight = 5  # Give more importance to UCS deviation from target

# Initialize variables to keep track of the best solution
best_score = float('inf')
best_solution = None
best_index = -1

# Iterate over all solutions and calculate a score based on the objectives
for i in range(len(res.X)):
    mix_design = res.X[i]
    co2 = model_co2.predict(mix_design.reshape(1, -1))[0]  # Predict CO₂ using model_co2
    cost = model_cost.predict(mix_design.reshape(1, -1))[0]  # Predict cost using model_cost
    ucs_pred = model_ucs.predict(mix_design.reshape(1, -1))[0]  # Predict UCS using model_ucs
    ucs_deviation = abs(ucs_pred - UCS_target)
    
    # Calculate a weighted score
    score = (co2_weight * co2) + (cost_weight * cost) + (ucs_deviation_weight * ucs_deviation)
    
    # Check if this solution is the best one so far
    if score < best_score:
        best_score = score
        best_solution = mix_design
        best_index = i

# Display the best solution and its details
print(f"Best solution index: {best_index + 1}")
print("Mix Design (Features):", best_solution)
print("Predicted CO₂:", model_co2.predict(best_solution.reshape(1, -1))[0])
print("Predicted Cost:", model_cost.predict(best_solution.reshape(1, -1))[0])
print("Predicted UCS:", model_ucs.predict(best_solution.reshape(1, -1))[0])
print("Score:", best_score)


Best solution index: 32
Mix Design (Features): [7.78804288e+02 3.83225047e+01 7.68791416e-01 1.09604906e+00
 2.72017240e+00 7.93463623e-01 5.70117913e-01 1.23713316e+03
 5.31804629e+02 1.09327683e+02 9.80915905e+00 7.69432404e+01
 3.82760520e+00 7.36083689e-01 7.93952530e+02 4.06893330e+02
 3.40010340e+01 3.66584567e+02 3.96923910e-01 2.16206836e+01
 6.78658426e+01 5.21679037e-02 3.16623011e+01 5.80692420e+01
 1.42430306e+01]
Predicted CO₂: 218.0669
Predicted Cost: 56.597603
Predicted UCS: 49.995117
Score: 274.6889114379883
